In [94]:
import pandas as pd
import numpy as np
import math
import json
import yaml

from itertools import combinations
from collections import Counter as ctr

In [95]:
df = pd.read_csv('100kindex.csv')

In [96]:
df.head()

,Unnamed: 0,word,DF
0,0,striking,"{351: 1, 969: 1, 1111: 1, 3902: 1, 4049: 1, 52..."
1,1,anxiety,"{1458: 1, 2235: 1, 5341: 1, 8121: 1, 10582: 1,..."
2,2,ee,"{1: 2, 3: 8, 7: 16, 9: 1, 10: 10, 12: 4, 13: 2..."
3,3,forgave,"{17495: 3, 22451: 1, 24795: 1, 29181: 1, 29880..."
4,4,kick,"{287: 1, 628: 3, 959: 1, 1070: 1, 1589: 1, 164..."


In [97]:
def idf(str_dict):
    word_dict = yaml.load(str_dict)
    N = 1662756
    n_w = len(word_dict)
    return (math.log(N/n_w,2))

In [98]:
df['idf'] = df['DF'].apply(idf)

In [99]:
df.head()

,Unnamed: 0,word,DF,idf
0,0,striking,"{351: 1, 969: 1, 1111: 1, 3902: 1, 4049: 1, 52...",11.734408
1,1,anxiety,"{1458: 1, 2235: 1, 5341: 1, 8121: 1, 10582: 1,...",13.620751
2,2,ee,"{1: 2, 3: 8, 7: 16, 9: 1, 10: 10, 12: 4, 13: 2...",4.836513
3,3,forgave,"{17495: 3, 22451: 1, 24795: 1, 29181: 1, 29880...",15.417218
4,4,kick,"{287: 1, 628: 3, 959: 1, 1070: 1, 1589: 1, 164...",10.572388


In [103]:
df.to_csv('complete_index.csv')

In [7]:
def frequency(w,d):
    return d.count(w)

In [76]:
def get_candidate_resources(query=[]):
    CR = []
    out = False
    count = 0
    no_repeats = []  #don't want to have the same documents repeated when doing combinations
    for doc in docs:
        if set(query).issubset(doc):
            count += 1
            CR.append(doc)
            no_repeats.append(doc)
        if count == 50:
            break
    if count<50:
        combs = combinations(query, len(query)-1)
        for c in list(combs):
            for doc in docs:
                if doc in no_repeats: continue
                if set(c).issubset(doc):
                    count+=1
                    CR.append(doc)
                    no_repeats.append(doc)
                if count==50:
                    out = True
                    break
            if out==True: break 
            else: continue
    return CR

In [8]:
def term_freq(w,d):
    c = ctr(d)
    max_showing_word = max(c.values())
    return frequency(w,d)/max_showing_word

In [21]:
def relevance_ranking(q, d):
    rel_score = 0
    for word in q:
        term_freq(word,d) * idf(w)
        rel_score += result
    return rel_score

In [35]:
string_query = "i really like dogs"
query = string_query.split()
query

['i', 'really', 'like', 'dogs']

In [52]:
doc1 = "dogs are fantastic we all really like dogs who would not like dogs i just love them"
doc1 = doc1.split()

doc2 = "this is a completely and way off document should not appear on CR"
doc2 = doc2.split()

doc3 = "dogs are so amazing, i like them"
doc3 = doc3.split()

docs = []
docs.extend([doc1, doc2, doc3])
#list_of_docs

In [53]:
cand_resources = get_candidate_resources(query)

In [54]:
cand_resources

[['dogs',
  'are',
  'fantastic',
  'we',
  'all',
  'really',
  'like',
  'dogs',
  'who',
  'would',
  'not',
  'like',
  'dogs',
  'i',
  'just',
  'love',
  'them'],
 ['dogs', 'are', 'so', 'amazing,', 'i', 'like', 'them']]

In [ ]:
rel_ranking_map = {}
for resource in cand_resources:
    

## I need the results from cand_resources to be the index of the doc and not the doc itself